# Lista 2


In [1]:
import numpy as np
import numpy as np
from scipy.linalg import solve

## Eliminacja Gaussa 


In [2]:
def gaussian_elimination(A, b):
    """
    Eliminacja Gaussa z częściowym wyborem elementu podstawowego.
    Rozwiązuje układ A x = b.
    """
    A = A.astype(float).copy()
    b = b.astype(float).copy()

    n = len(A)

    # Eliminacja w przód
    for k in range(n):
        # wybór pivota (wiersz z max |A[i,k]| od k w dół)
        pivot = k + np.argmax(np.abs(A[k:, k]))
        if abs(A[pivot, k]) < 1e-12:
            raise ValueError("Macierz osobliwa lub prawie osobliwa.")

        # zamiana wierszy
        if pivot != k:
            A[[k, pivot]] = A[[pivot, k]]
            b[[k, pivot]] = b[[pivot, k]]

        # zerowanie poniżej przekątnej
        for i in range(k + 1, n):
            factor = A[i, k] / A[k, k]
            A[i, k:] -= factor * A[k, k:]
            b[i]     -= factor * b[k]

    # podstawianie wsteczne
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]

    return x


# Wyznacznik macierzy

In [3]:
def determinant_gauss(A):
    """
    Oblicza wyznacznik macierzy A metodą eliminacji Gaussa
    z częściowym wyborem elementu podstawowego.
    """
    A = A.astype(float).copy() #copy, żeby nie modyfikować oryginalnej macierzy
    n = A.shape[0] # zakładamy liczbe kolumn = liczbie wierszy

    det_sign = 1.0  # zmienia się przy zamianie wierszy, znak wyznacznika

    if A.shape[0] != A.shape[1]:
        raise ValueError("Macierz musi być kwadratowa, aby mieć wyznacznik.")
    
    for k in range(n):
        # wybór pivota (wiersz z maksymalnym |A[i,k]| od k w dół)
        pivot = k + np.argmax(np.abs(A[k:, k]))
        if abs(A[pivot, k]) < 1e-12:
            # pivot ~ 0 ⇒ macierz osobliwa ⇒ det = 0
            return 0.0
# W efekcie pivot = numer wiersza z największą wartością bezwzględną w kolumnie k

        # zamiana wierszy (zmiana znaku wyznacznika)
        if pivot != k:
            A[[k, pivot]] = A[[pivot, k]]
            det_sign *= -1.0

        # eliminacja poniżej pivota
        for i in range(k + 1, n):
            factor = A[i, k] / A[k, k] #współczynnik eliminacji - ile razy wiersz k trzeba dodać/odjąć od wiersza i, żeby wyzerować element w kolumnie k
            A[i, k:] -= factor * A[k, k:]

    # teraz A jest w postaci trójkątnej górnej,
    # wyznacznik = znak * iloczyn elementów na przekątnej
    det = det_sign
    for i in range(n):
        det *= A[i, i]

    return det

# Macierz Odwrotna

In [4]:
def inverse_via_gauss(A):
    """
    Oblicza macierz odwrotną A^{-1} rozwiązując A x = e_i
    dla kolejnych wektorów jednostkowych e_i.
    """
    n = A.shape[0]
    A_inv = np.zeros_like(A, dtype=float) #tworzy nową macierz o takim samym kształcie jak A, wypełnioną zerami, typu float. 

    for i in range(n):
        e = np.zeros(n) #tworzysz wektor zerowy długości n
        e[i] = 1.0 # ustawia jedynkę na pozycji i
        col = gaussian_elimination(A, e)
        A_inv[:, i] = col  

    return A_inv

# Czy macierz jest trójdiagonalna?

In [5]:
def is_tridiagonal(M, tol=1e-12): 
    """
    Sprawdza, czy macierz jest (w przybliżeniu) trójdiagonalna,
    tzn. elementy z |i-j| > 1 są (numerycznie) równe 0.
    """
    n = M.shape[0]
    for i in range(n):
        for j in range(n):
            if abs(i - j) > 1 and abs(M[i, j]) > tol:
                return False
    return True

## zad 4


In [6]:
A = np.array([[0, 0, 2,  1,  2],
              [0, 1, 0,  2, -1],
              [1, 2, 0, -2,  0],
              [0, 0, 0, -1,  1],
              [0, 1,-1,  1, -1]], dtype=float)

b = np.array([1, 1, -4, -2, -1], dtype=float)

x_gauss = gaussian_elimination(A.copy(), b.copy())
x_scipy = solve(A, b)

print("Rozwiązanie wykonane za pomocą napisanej funkcji eliminacji Gaussa:", x_gauss)
print("Rozwiązanie wykonane za pomocą biblioteki scipy:", x_scipy)



Rozwiązanie wykonane za pomocą napisanej funkcji eliminacji Gaussa: [ 2. -2.  1.  1. -1.]
Rozwiązanie wykonane za pomocą biblioteki scipy: [ 2. -2.  1.  1. -1.]


## zad 5

In [7]:
# Zadanie 5 – wyznaczenie współczynników wielomianu
# y = a0 + a1 x + a2 x^2 + a3 x^3 + a4 x^4
# przechodzącego przez punkty (0,-1), (1,1), (3,3), (5,2), (6,-2)

x_pts = np.array([0, 1, 3, 5, 6], dtype=float)
y_pts = np.array([-1, 1, 3, 2, -2], dtype=float)

# Macierz Vandermonde'a: wiersz = [1, x, x^2, x^3, x^4]
A = np.vstack([x_pts**0, x_pts**1, x_pts**2, x_pts**3, x_pts**4]).T
b = y_pts

coef_gauss = gaussian_elimination(A.copy(), b.copy())
coef_scipy = solve(A, b)

print("Współczynniki (Gauss):", coef_gauss)
print("Współczynniki (scipy):", coef_scipy)


Współczynniki (Gauss): [-1.          2.68333333 -0.875       0.21666667 -0.025     ]
Współczynniki (scipy): [-1.          2.68333333 -0.875       0.21666667 -0.025     ]


## zad 6 


In [8]:
if __name__ == "__main__":
    A = np.array([
        [3.50, 2.77, -0.76, 1.80],
        [-1.80, 2.68,  3.44, -0.09],
        [0.27, 5.07,  6.90,  1.61],
        [1.71, 5.45,  2.68,  1.71],
    ])
    b = np.array([7.31, 4.23, 13.85, 11.55])

    # Rozwiązanie układu A x = b
    x = gaussian_elimination(A, b)
    print("Rozwiązanie x =",  x)

  # porównanie z wbudowanym solve
    x_scipy = solve(A, b)
    print("\nRozwiązanie z scipy.linalg.solve (do porównania):")
    print(x_scipy)

   # Wyznacznik macierzy A
    detA = determinant_gauss(A)
    print("\ndet(A) =")
    print(detA)

    # Macierz Ax
    Ax = np.dot(A, x)
    print("\nAx =")
    print(Ax)

    reszta = b - Ax
    print("\nr =")
    print(reszta)

Rozwiązanie x = [1. 1. 1. 1.]

Rozwiązanie z scipy.linalg.solve (do porównania):
[1. 1. 1. 1.]

det(A) =
-0.22579734000000884

Ax =
[ 7.31  4.23 13.85 11.55]

r =
[-8.88178420e-16  0.00000000e+00  1.77635684e-15  1.77635684e-15]


## zad 7

In [9]:
# Zadanie 7 – rozwiąż układ A x = b

A = np.array([
    [10,  -2, -1,  2,  3,  1, -4,  7],
    [ 5,  11,  3, 10, -3,  3,  3, -4],
    [ 7,  12,  1,  5,  3, -12,  2,  3],
    [ 8,   7, -2,  1,  3,  2,  2,  4],
    [ 2, -15, -1,  1,  4, -1,  8,  3],
    [ 4,   2,  9,  1, 12, -1,  4,  1],
    [-1,   4, -7, -1,  1,  1, -1, -3],
    [-1,   3,  4,  1,  3, -4,  7,  6]
], dtype=float)

b = np.array([0, 12, -5, 3, -25, -26, 9, -7], dtype=float)

x_gauss = gaussian_elimination(A.copy(), b.copy())
x_scipy = solve(A, b)

print("Rozwiązanie wykonane za pomocą napisanej funkcji eliminacji Gaussa:", x_gauss)
print("Rozwiązanie wykonane za pomocą biblioteki scipy:", x_scipy)


Rozwiązanie wykonane za pomocą napisanej funkcji eliminacji Gaussa: [-1.  1. -1.  1. -1.  1. -1.  1.]
Rozwiązanie wykonane za pomocą biblioteki scipy: [-1.  1. -1.  1. -1.  1. -1.  1.]


## Zad 8

In [10]:
if __name__ == "__main__":
    A = np.array([
        [ 2, -1,  0,  0,  0,  0],
        [-1,  2, -1,  0,  0,  0],
        [ 0, -1,  2, -1,  0,  0],
        [ 0,  0, -1,  2, -1,  0],
        [ 0,  0,  0, -1,  2, -1],
        [ 0,  0,  0,  0, -1,  5]
    ], dtype=float)

    A_inv = inverse_via_gauss(A) 
    np.set_printoptions(precision=4, suppress=True) # pr - do 4 miejsc po przecinku, sup - ukrywa liczby bardzo małe i zwraca zero
    print("A^{-1} =")
    print(A_inv)

    print("\nCzy A jest trójdiagonalna?")
    print(is_tridiagonal(A))

    print("\nCzy A^{-1} jest trójdiagonalna?")
    print(is_tridiagonal(A_inv))



A^{-1} =
[[0.84 0.68 0.52 0.36 0.2  0.04]
 [0.68 1.36 1.04 0.72 0.4  0.08]
 [0.52 1.04 1.56 1.08 0.6  0.12]
 [0.36 0.72 1.08 1.44 0.8  0.16]
 [0.2  0.4  0.6  0.8  1.   0.2 ]
 [0.04 0.08 0.12 0.16 0.2  0.24]]

Czy A jest trójdiagonalna?
True

Czy A^{-1} jest trójdiagonalna?
False


## Zad 9

In [11]:
if __name__ == "__main__":
    A = np.array([
        [ 1,  3, -9,  6,  4],
        [ 2, -1,  6,  7,  1],
        [ 3,  2, -3, 15,  5],
        [ 8, -1,  1,  4,  2],
        [11,  1, -2, 18,  7]
    ], dtype=float)

    A_inv = inverse_via_gauss(A)
    np.set_printoptions(precision=4, suppress=True)
    print("A^{-1} =")
    print(A_inv)



ValueError: Macierz osobliwa lub prawie osobliwa.